# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']
A. Gould  ->  A. Gould  |  ['A. Gould']
S. Li  ->  S. Li  |  ['S. Li']


M. Habouzit  ->  M. Habouzit  |  ['M. Habouzit']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 68 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2306.14990


extracting tarball to tmp_2306.14990...

 done.
Retrieving document from  https://arxiv.org/e-print/2306.15037


extracting tarball to tmp_2306.15037...

...

/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:730: LatexWarning: Could not extract abstract from tmp_2306.15037/yee_draft3.1.tex
  warnings.warn(LatexWarning(f"Could not extract abstract from {self.main_file}"))


list index out of range
Retrieving document from  https://arxiv.org/e-print/2306.15346


extracting tarball to tmp_2306.15346...

 done.
Retrieving document from  https://arxiv.org/e-print/2306.15527


extracting tarball to tmp_2306.15527... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.15037-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.15037) | **The Scientific Discovery Space for the Roman Galactic Bulge Time Domain  Survey**  |
|| J. C. Yee, <mark>A. Gould</mark> |
|*Appeared on*| *2023-06-28*|
|*Comments*| *10 pages, submitted to Roman Core Community Survey white paper call*|
|**Abstract**| Maximizing the scientific return of Roman requires focusing on the scientific discovery space opened up by Roman relative to the ground: i.e., planets in wide orbits (log s > 0.4), the smallest mass-ratio planets (log q < -4.5), and free-floating planet candidates (especially those with thetaE < 1 uas). However, capitalizing on that leverage requires not just detecting such planets but characterizing them sufficiently that they can be used in a statistical analysis. In particular, the signals from all three categories are all prone to light curve degeneracies that may lead to ambiguities in the planet mass-ratio q, separation s, and the size of the source rho (used to measure thetaE and constrain the host mass). Bound planets may also have light curves that are degenerate with models that include a second source rather than a planet. The most immediate need for designing the Roman Galactic Bulge Time Domain Survey is a detailed simulation of wide-orbit and small planetary perturbations to investigate how well the planet perturbations will be characterized. These investigations and related trade-studies must be done in order to maximize Roman's ability to take advantage of new parameter space. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.14990-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.14990) | **Massive Black Hole Binaries as LISA Precursors in the Roman High  Latitude Time Domain Survey**  |
|| Z. Haiman, et al. -- incl., <mark>M. Habouzit</mark> |
|*Appeared on*| *2023-06-28*|
|*Comments*| *White Paper for the Nancy Grace Roman Space Telescope's Core Community Surveys (this https URL)*|
|**Abstract**| With its capacity to observe $\sim 10^{5-6}$ faint active galactic nuclei (AGN) out to redshift $z\approx 6$, Roman is poised to reveal a population of $10^{4-6}\, {\rm M_\odot}$ black holes during an epoch of vigorous galaxy assembly. By measuring the light curves of a subset of these AGN and looking for periodicity, Roman can identify several hundred massive black hole binaries (MBHBs) with 5-12 day orbital periods, which emit copious gravitational radiation and will inevitably merge on timescales of $10^{3-5}$ years. During the last few months of their merger, such binaries are observable with the Laser Interferometer Space Antenna (LISA), a joint ESA/NASA gravitational wave mission set to launch in the mid-2030s. Roman can thus find LISA precursors, provide uniquely robust constraints on the LISA source population, help identify the host galaxies of LISA mergers, and unlock the potential of multi-messenger astrophysics with massive black hole binaries. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.15346-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.15346) | **Episodic Accretion in Protostars -- An ALMA Survey of Molecular Jets in  the Orion Molecular Cloud**  |
|| S. Dutta, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-06-28*|
|*Comments*| *Submitted to Journal; 27 pages, 15 Figures and additional Appendix material*|
|**Abstract**| Protostellar outflows and jets are almost ubiquitous characteristics during the mass accretion phase, and encode the history of stellar accretion, complex-organic molecule (COM) formation, and planet formation. Episodic jets are likely connected to episodic accretion through the disk. Despite the importance, there is a lack of studies of a statistically significant sample of protostars via high-sensitivity and high-resolution observations. To explore episodic accretion mechanisms and the chronologies of episodic events, we investigated 42 fields containing protostars with ALMA observations of CO, SiO, and 1.3\,mm continuum emission. We detected SiO emission in 21 fields, where 19 sources are driving confirmed molecular jets with high abundances of SiO. Jet velocities, mass-loss rates, mass-accretion rates, and periods of accretion events are found to be dependent on the driving forces of the jet (e.g., bolometric luminosity, envelope mass). Next, velocities and mass-loss rates are positively correlated with the surrounding envelope mass, suggesting that the presence of high mass around protostars increases the ejection-accretion activity. We determine mean periods of ejection events of 20$-$175 years for our sample, which could be associated with perturbation zones of $\sim$ 2$-$25\,au extent around the protostars. Also, mean ejection periods are anti-correlated with the envelope mass, where high-accretion rates may trigger more frequent ejection events. The observed periods of outburst/ejection are much shorter than the freeze-out time scale of the simplest COMs like CH$_3$OH, suggesting that episodic events largely maintain the ice-gas balance inside and around the snowline. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2306.15527-b31b1b.svg)](https://arxiv.org/abs/arXiv:2306.15527) | **Discovery and Characterization of Galactic-scale Dual Supermassive Black  Holes Across Cosmic Time**  |
|| Y. Shen, et al. -- incl., <mark>M. Habouzit</mark> |
|*Appeared on*| *2023-06-28*|
|*Comments*| *Roman Core Community Survey White Paper, focusing on the High Latitude Wide Area Survey*|
|**Abstract**| The hierarchical structure formation paradigm predicts the formation of pairs of supermassive black holes in merging galaxies. When both (or one) members of the SMBH pair are unobscured AGNs, the system can be identified as a dual (or offset) AGN. Quantifying the abundance of these AGN pairs as functions of separation, redshift and host properties is crucial to understanding SMBH formation and AGN fueling in the broad context of galaxy formation. The High Latitude Wide Area Survey with Roman, with its unprecedented combination of sensitivity, spatial resolution, area and NIR wavelength coverage, will revolutionize the study of galactic-scale environments of SMBH pairs. This white paper summarizes the science opportunities and technical requirements on the discovery and characterization of SMBH pairs down to galactic scales (i.e., less than tens of kpc) over broad ranges of redshift (1<z<7) and luminosity (Lbol>1E42 erg/s). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2306.15037.md
    + _build/html/tmp_2306.15037/./ulens_disc_by_year.png
    + _build/html/tmp_2306.15037/./yee_layout_7f_3_covfac_52_3_sensitivity_by_year.png
    + _build/html/tmp_2306.15037/./OB161195_lc.png
    + _build/html/tmp_2306.15037/./snrrat-eps-converted-to.png
    + _build/html/tmp_2306.15037/./matchcum-eps-converted-to.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\WF}{{\em Roman}}$
$\newcommand{\wangevent}{OGLE-2018-BLG-0383}$
$\newcommand{\wband}{W146-band}$
$\newcommand{\zband}{Z087-band}$
$\newcommand{\bdv}[1]{\mbox{\boldmath#1}}$
$\newcommand{\bpi}{{\bdv\pi}}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# The Scientific Discovery Space for the $\WF$  Galactic Bulge Time Domain Survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2306.15037-b31b1b.svg)](https://arxiv.org/abs/2306.15037)<mark>Appeared on: 2023-06-28</mark> -  _10 pages, submitted to Roman Core Community Survey white paper call_

</div>
<div id="authors">

J. C. Yee, <mark>A. Gould</mark>

</div>
<div id="abstract">

**Abstract:** Maximizing the scientific return of Roman requires focusing on the scientific discovery space opened up by Roman relative to the ground: i.e., planets in wide orbits (log s > 0.4), the smallest mass-ratio planets (log q < -4.5), and free-floating planet candidates (especially those with thetaE < 1 uas). However, capitalizing on that leverage requires not just detecting such planets but characterizing them sufficiently that they can be used in a statistical analysis. In particular, the signals from all three categories are all prone to light curve degeneracies that may lead to ambiguities in the planet mass-ratio q, separation s, and the size of the source rho (used to measure thetaE and constrain the host mass). Bound planets may also have light curves that are degenerate with models that include a second source rather than a planet. The most immediate need for designing the Roman Galactic Bulge Time Domain Survey is a detailed simulation of wide-orbit and small planetary perturbations to investigate how well the planet perturbations will be characterized. These investigations and related trade-studies must be done in order to maximize Roman's ability to take advantage of new parameter space. 

</div>

<div id="div_fig1">

<img src="tmp_2306.15037/./ulens_disc_by_year.png" alt="Fig1.1" width="50%"/><img src="tmp_2306.15037/./yee_layout_7f_3_covfac_52_3_sensitivity_by_year.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** $\WF$  Galactic Bulge Time Domain Survey discovery space compared to microlensing discoveries over time (NASA Exoplanet Archive, accessed 5/19/23). Top, left axis (magenta): number of microlensing planets published each year. A publication rate of 40 planets/year from the ground is assumed for future years. Top, right axis (black): cumulative number of published microlensing planets (solid=actual, dotted=projected). The black dashed lines indicate the approximate times when white papers were being prepared for the 2010 and 2020 decadal surveys and the start of the $\WF$  Galactic  Bulge Time Domain Survey. Bottom \citep{Penny19}: squares = known planet distribution at the end of 2008 (magenta), 2018 (cyan), and now (black) compared to the Solar System planets and simulated $\WF$  planet yield (gray points). The blue line indicates the expected $\WF$ , sensitivity limit. (*fig:discovery_space*)

</div>
<div id="div_fig2">

<img src="tmp_2306.15037/./OB161195_lc.png" alt="Fig5" width="100%"/>

**Figure 5. -** Small mass ratio planets can produce "bump"-like perturbations that may be subject to degeneracies such as the "central-resonant" degeneracy. This example shows two possible models for the planet in OGLE-2016-BLG-1195. Similar to potential degeneracies for wide-orbit planets, the RGBS cadence will need to be high enough to distinguish between various possible models. \citep[Figure from ][]{Gould23_ob1195} (*fig:ob1195*)

</div>
<div id="div_fig3">

<img src="tmp_2306.15037/./snrrat-eps-converted-to.png" alt="Fig3.1" width="50%"/><img src="tmp_2306.15037/./matchcum-eps-converted-to.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** $\WF$'s primary advantage over ground-based surveys comes at low magnification. For $A = 1$, $\WF$  has a signal-to-noise advantage of $\sim 5$--$12$ over the KMTNet, but at $A=100$ that advantage is a relatively modest factor of $\sim 2$.
	Bottom: Absolute magnitudes of RGBS sources compared to KMTNet sources at the same cumulative fraction; e.g., half of all RGBS sources will be $M_I < 6$ whereas half of all KMTNet sources are $M_I < 3.25$. Top: relative signal-to-noise ratio between $\WF$  and KMTNet as a function of cumulative fraction of sources and magnification. Middle: same as top, but as a function of $\WF$  absolute source magnitude, for which the corresponding KMTNet source magnitude can be found from the bottom panel; e.g., for a $\WF$  source with $M_I = 6$, the corresponding KMTNet source will be $M_I = 3.25$, and for such sources, $\WF$  has a SNR advantage of $\sim 11.5$ over KMTNet at $A=1$, but only $\sim 3$ at $A=100$. Figures from \citet{Gould23_kb2397}. (*fig:wf_kmt*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2306.15037"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

222  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
